In [130]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [131]:
df = pd.read_csv('한국소비자원 생필품 및 서비스 가격 정보_20220128.csv', encoding = 'cp949')
df.head()

,상품명,조사일,판매가격,판매업소,제조사,세일여부,원플러스원
0,머거본 꿀땅콩(135g),2022-01-14,3600,CU(본사),머거본,N,N
1,머거본 꿀땅콩(135g),2022-01-14,2190,롯데슈퍼안양점,머거본,N,N
2,허니버터아몬드(210g),2022-01-14,5990,롯데슈퍼김포전원점,길림양행,N,N
3,허니버터아몬드(210g),2022-01-14,5990,롯데슈퍼길동점,길림양행,N,N
4,허니버터아몬드(210g),2022-01-14,5990,롯데슈퍼구평점,길림양행,N,N


In [132]:
prods = df['상품명'].unique()
prods

array(['머거본 꿀땅콩(135g)', '허니버터아몬드(210g)', '백설 소면(900g)', '옛날국수 소면(900g)',
       '고향만두(1.08kg)', '비비고 왕교자(1.05kg)', '생야채돼지고기물만두(800g)',
       '백설 자른 햇당면(300g)', '옛날 자른당면(300g)', '국산콩두부 부침용(380g)',
       '국산콩두부 찌개용(380g)', '행복한콩 부침두부(300g)', '행복한콩 찌개두부(300g)',
       '삼양라면(5개입)', '신라면(5개입)', '진라면 순한맛(5개입)', '곰표 밀가루 중력다목적용(1kg)',
       '백설 찰밀가루(1kg)', '백설 5가지 재료 부침가루(1kg)', '오뚜기 부침가루(1kg)',
       '삼립호떡 미니꿀호떡(16개입)', '정통크림빵(3개입)', '보노 콘스프(3개입)',
       '보노 포르치니버섯스프(3개입)', '오뚜기 크림스프(80g)', '스페셜K 오리지널(480g)',
       '켈로그 콘푸로스트(600g)', '포스트 콘푸라이트(600g)', '오뚜기 간편 미역국(5개입)',
       '오뚜기 간편 북엇국(5개입)', '3분 쇠고기짜장(200g)', '3분 쇠고기카레(200g)',
       '오뚜기 3분 백세카레 약간 매운맛(200g)', '청정원 카레여왕 비프카레(160g)', '오뚜기밥(210g)',
       '오뚜기밥(3개입)', '오뚜기밥(6개입) ', '햇반(210g)', '햇반(3개입)', '햇반(6개입)',
       '동원 양반 쇠고기죽(287.5g)', '동원 양반 전복죽(287.5g)', '비비고 소고기죽(280g)',
       '비비고 전복죽(280g)', '농심 신라면 큰사발면(114g)', '오뚜기 참깨라면 컵(110g)',
       '팔도 왕뚜껑(110g)', '오뚜기 맛있는 쇠고기미역국밥(162g)', '오뚜기 컵밥 제육덮밥(310g)',
       '햇반 컵반 고추장 제

In [133]:
## 유니크 품목

products = []
for product in prods:
    products.append(product)
products

['머거본 꿀땅콩(135g)',
 '허니버터아몬드(210g)',
 '백설 소면(900g)',
 '옛날국수 소면(900g)',
 '고향만두(1.08kg)',
 '비비고 왕교자(1.05kg)',
 '생야채돼지고기물만두(800g)',
 '백설 자른 햇당면(300g)',
 '옛날 자른당면(300g)',
 '국산콩두부 부침용(380g)',
 '국산콩두부 찌개용(380g)',
 '행복한콩 부침두부(300g)',
 '행복한콩 찌개두부(300g)',
 '삼양라면(5개입)',
 '신라면(5개입)',
 '진라면 순한맛(5개입)',
 '곰표 밀가루 중력다목적용(1kg)',
 '백설 찰밀가루(1kg)',
 '백설 5가지 재료 부침가루(1kg)',
 '오뚜기 부침가루(1kg)',
 '삼립호떡 미니꿀호떡(16개입)',
 '정통크림빵(3개입)',
 '보노 콘스프(3개입)',
 '보노 포르치니버섯스프(3개입)',
 '오뚜기 크림스프(80g)',
 '스페셜K 오리지널(480g)',
 '켈로그 콘푸로스트(600g)',
 '포스트 콘푸라이트(600g)',
 '오뚜기 간편 미역국(5개입)',
 '오뚜기 간편 북엇국(5개입)',
 '3분 쇠고기짜장(200g)',
 '3분 쇠고기카레(200g)',
 '오뚜기 3분 백세카레 약간 매운맛(200g)',
 '청정원 카레여왕 비프카레(160g)',
 '오뚜기밥(210g)',
 '오뚜기밥(3개입)',
 '오뚜기밥(6개입) ',
 '햇반(210g)',
 '햇반(3개입)',
 '햇반(6개입)',
 '동원 양반 쇠고기죽(287.5g)',
 '동원 양반 전복죽(287.5g)',
 '비비고 소고기죽(280g)',
 '비비고 전복죽(280g)',
 '농심 신라면 큰사발면(114g)',
 '오뚜기 참깨라면 컵(110g)',
 '팔도 왕뚜껑(110g)',
 '오뚜기 맛있는 쇠고기미역국밥(162g)',
 '오뚜기 컵밥 제육덮밥(310g)',
 '햇반 컵반 고추장 제육덮밥(250g)',
 '햇반 컵반 미역국밥(167g)',
 '비비고 사골곰탕(500g)',
 '비비고 육개장(

In [134]:
## 유니크 품목을 낮은 가격순으로 검색 했을 때 나오는 모든 제품들 크롤링


from collections import deque
sample_ = deque()
for prod in prods:
    url = 'https://www.coupang.com/np/search?rocketAll=false&q={}&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&searchProductCount=326&component=&rating=0&sorter=salePriceAsc&listSize=36'
    url = url.format(prod)
    
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'}
    
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    lis = soup.select(' a > dl > dd > div')
    
    for li in lis:
        try:
                sample_.appendleft([li.select('div.name')[0].text,
                        li.select('strong.price-value')[0].text])
                print(li.select('div.name')[0].text)
                print(li.select('strong.price-value')[0].text)
        except:
            pass
        

머거본 알땅콩(캔) 135g, 1개
1,890
[BIGHAND] 머거본 알땅콩 캔 135g, 1개
1,970
머거본 꿀땅콩 135g 식품 > 신선식품 건과/견과류 견과류 일반땅콩, 1개입
1,980
[머거본] 꿀땅콩 135g, 단품
2,100
상세설명참고 머거본 꿀땅콩 캔 135g
2,300
[머거본] 꿀땅콩 135g 1캔
2,300
머거본 꿀땅콩 135g, 상세페이지 참조
2,330
머거본 꿀땅콩 캔 135g 식품 > 스낵/간식 원물간식 기타원물간식, 1, 1개
2,390
머거본 알땅콩 135g x 1캔
2,420
머거본 꿀땅콩 135g 너트 피넛 캔 안주 허니 맛땅콩, 1N5R_머거본 꿀땅콩 135g
3,000
머거본 허니 꿀땅콩 135g 피넛 견과류 마른 안주 간식 군것질 주전부리
3,080
머거본 원하프꿀땅콩, 135g
3,310
샘표 질러 부드러운 육포 30G, 단품
3,500
(맛나)꿀땅콩 135G/머거본, 1개
3,530
무배_꿀땅콩(머거본 135g)/식당, 1
3,680
머거본 꿀땅콩 135g, 상세페이지 참조
4,800
☞고객대만족↗¶ 견과류 아버지술 소주안주 아빠술 꿀땅콩 맥주안주 머거본 (k‡f*H) 135g, 1개, ♬본 상품 선택하기_Happys™
5,250
¶⊙핫딜_A+ 아버지 맥주 술안주 견과류 소주 FK 안주 꿀땅콩 머거본 (Mf!♩x) 135g, 1개, ☞ 해당 상품 선택하기_DoDream™
5,290
머거본 꿀땅콩 135g, 3개입
5,310
Happy 꿀땅콩(머거본 135g) 건강식품 농산물 부모님선물, 해피니스하우스 1, 해피니스하우스 본상품선택
5,400
머거본 후렌치 후라이드 아몬드 135g, 본상품선택
5,900
꿀땅콩(머거본 135g) 대용량 업소용 식당용 납품용
7,020
꿀땅콩 캔 135g X 3 견과류 와인 맥주안주, 1IYM_꿀땅콩 캔 135g X3
7,200
머거본 꿀땅콩 135g 마른안주 업소용안주, 1
8,210
꿀땅콩 머거본 135g
8,860
머거본 칼몬드 혼합견과 100g + 새우몬드 80g + 꿀땅

생야채 돼지고기 물만두 800g (400g 2개입) 3봉, 3개
21,630
CJ제일제당 생야채 돼지고기 생 물만두 (냉동), 400g, 2봉
6,930
씨제이제일제당 백설 자른햇당면 300g x 1개, 단품
2,980
[CJ] 백설 자른햇당면 300G x 3개, 단품
7,440
보니또댕댕이 사각큐브 강아지 간식 300g, 양고기, 5개
8,130
보니또댕댕이 반려동물 사사미 300g, 오리고기 빼로로, 5개
9,000
보니또댕댕이 양고기 윙 반려동물 간식 300g, 5개
11,200
씨제이제일제당 백설 자른햇당면 300g x 5개, 단품
14,400
씨제이제일제당 백설 자른햇당면 300g x 10개, 단품
28,800
보니또댕댕이 반려동물 사사미 300g, 소고기 빼로로, 5개
8,500
오뚜기옛날 자른당면, 100g, 1개
1,200
화펑 샤부샤부용당면 300g, 단품, 단품
2,600
오뚜기 자른당면300g, 상세페이지 참조, 루쿠 본상품선택
3,260
k1생감자+고구마100%면(300gx1개)1분 바로 조리당면 물에 불리지 않는당면 칼면 넙적당면[선택형], 감자면300g + 1개, 8mm넙적면
3,430
k1생고구마+감자100%면(300gx1개)1분 바로 조리당면 물에 불리지 않는 고구마당면 칼면 넙적당면[선택형], 감자면300gx 1개, 5mm칼면
3,430
k1생고구마+감자100%면(300gx1개)1분 바로 조리당면 물에 불리지 않는 고구마당면 칼면 넙적당면[선택형], 감자면300g + 1개, 18m넙적면
3,430
K1감자면+K1고구마100%면300gx1개1분 바로 조리당면 물에불리지 않는 당면/칼면 넙적면[선택형], 감자면300gx 1개, 18mm넙적면
3,430
오뚜기옛날 자른당면, 300g, 1개
3,520
예이니종합물류 오뚜기 옛날 당면 1개(300g*1개) 잡채메밀비빔칼국수 즉석간편요리식품, 300g, 1개
3,790
(무)오뚜기 옛날당면300g
3,800
오뚜기 옛날당면300g
3,930
오뚜기 옛날자른당면300g오뚜기, 옛날자른당면30

곰표 밀가루 1kg 중력다목적용 곰표 밀가루 중력 분, 단품
1,780
곰표 밀가루 중력다목적용, 1kg, 1개
1,910
(J.R.M.)곰표밀가루(중력다목적용)1kg9999EA, 쿠팡 1, 쿠팡 본상품선택
2,150
곰표밀가루(중력다목적용)1kg, 상세페이지 참조
2,190
곰표밀가루 중력다목적용(1kgx3입)
3,920
EM018 곰표 밀가루 1kg 중력다목적용, 상세페이지 참조
4,710
곰표밀가루(중력다목적용)1kg 1개 / 2개 / 5개 /20개, 밀가루 1kg x 1개
5,900
곰표 밀가루 중력다목적용, 1kg, 2개
6,500
곰표 밀가루 중력다목적용, 1kg, 3개
7,100
다목적용 곰표 중력 밀가루 1kg 업소용 식당용 식자재 소스 양념 대용량, 본상품선택
6,460
대한제분 곰표 밀가루 1kg x 3 중력분 중력 다목적용 수제비 칼국수 만두 부침 무표백
7,970
곰표 밀가루 1kg X 3입 / 중력밀가루1등급
7,980
곰표밀가루(중력다목적용)1kg 1개 / 2개 / 5개 /20개, 밀가루 1kg x 2개
8,100
곰표 밀가루 중력다목적용, 1kg, 4개
9,700
곰표 밀가루 중력다목적용, 1kg, 5개
10,400
곰표 밀가루 중력다목적용, 1kg, 6개
11,500
곰표 밀가루 중력다목적용, 1kg, 7개
13,000
곰표 밀가루 중력다목적용, 1kg, 8개
14,300
곰표밀가루(중력다목적용)1kg 1개 / 2개 / 5개 /20개, 밀가루 1kg x 5개
14,500
곰표 밀가루 중력다목적용, 1kg, 9개
15,700
슈퍼츄 일 킬로그램 가득담은 소프트 져키 1kg, 소고기맛, 1개
15,900
슈퍼츄 일 킬로그램 가득담은 소프트 져키 1kg, 닭고기맛, 1개
17,150
인스팅트 전연령용 네이처스버라이어티 BE내추럴 독 키블 강아지 건식사료, 1kg, 닭
17,700
훔볼트 피오네르 스페셜티 블렌드 원두커피, 홀빈(분쇄안함), 1kg
19,000
BBB819 W/ 곰표밀가루 중력다목적용 1kgx10개 1박스, 단일 수량
21

오뚜기 양송이스프80g오뚜기, 04.양송이스프
3,910
오뚜기 크림 스프, 300g, 1개
4,440
오뚜기 크림 스프, 80g, 2개
5,520
오뚜기 크림스프 1kg, 1개
5,780
오뚜기 크림스프 1kg, 1개
5,780
[김멜헤이] 오뚜기 쇠고기스프 80g 300g 500g 1kg 대용량 소고기 스프 영양성분 비프 크림스프 업소용 식자재 쇠고기 수프
6,120
오뚜기 크림 스프, 80g, 7개
6,860
오뚜기 양송이 스프, 80g, 3개
6,890
오뚜기 크림 스프, 80g, 3개
7,090
오뚜기 야채스프 80g 3개
7,100
오뚜기 쇠고기 스프 80g x 2p + 크림 스프 80g x 2p, 1세트
8,470
오뚜기 크림 스프 80g x 2p + 양송이 스프 80g x 2p, 1세트
8,900
오뚜기 크림 스프, 80g, 5개
10,250
예이니식품 오뚜기 쇠고기 스프 6개(80gx6개) 양송이콘야채크림
10,600
예이니식품 오뚜기 스프 인기5종 각1개씩 세트(80gx5개) 양송이크림콘쇠고기야채, 80g, 5개
10,650
[로적마켓] 오뚜기 스프 5종 혼합세트 각1개씩(각80g) (쇠고기+양송이+옥수수+크림+야채), 1세트
10,950
오뚜기 양송이크림 스프 단호박크림스프 콘크림스프 베이컨감자스프 270g 4개 골라담기 빠른배송
11,210
오뚜기 크림 스프, 80g, 6개
11,240
오뚜기 옥수수스프 80gx6개입, 80g, 6개입
11,550
오뚜기 크림 스프 80g x 2p + 옥수수 스프 80g x 2p, 1세트
11,600
오뚜기 양송이 스프, 80g, 6개
11,780
오뚜기 크림스프 80gx10입, 아이스박스포장
13,080
오뚜기 크림 스프, 80g, 4개
12,990
오뚜기 크림 스프, 80g, 10개
13,480
오뚜기 크림 스프, 80g, 10개
13,500
오뚜기 쇠고기 스프 80g x 2p + 크림 스프 80g x 2p, 2세트
13,770
양송이 크림스프 80g X 7 간편식 즉석스프
14,580
오뚜기 크림 스

3분 쇠고기카레 200g 간편식 즉석 카레, 상세페이지 참조
1,050
오뚜기 3분 쇠고기짜장 200g/즉석식품/국/덮밥/덥밥/소스/카레/짜장/3분요리 식품 > 가공/즉석식품 즉석밥/간편조리 카레/짜장/덮밥 즉석카레, 1
1,090
오뚜기 3분 쇠고기간짜장 200g 식품 > 가공/즉석식품 즉석밥/간편조리 카레/짜장/덮밥 즉석카레, 1
1,100
오뚜기 3분쇠고기카레200g
1,110
오뚜기 3분 쇠고기카레 200g
1,110
오뚜기3분 쇠고기카레, 200g, 1개
1,180
오뚜기 3분 쇠고기짜장 200g 식품 > 가공/즉석식품 즉석밥/간편조리 카레/짜장/덮밥 즉석카레, 1
1,200
오뚜기 3분쇠고기짜장200g
1,290
오뚜기 3분쇠고기카레200g, 쿠팡 1, 쿠팡 본상품선택
1,360
[ 오뚜기 3분 쇠고기 카레 200g 1개 ] 즉석 요리 식품 간편 음식 짜장 미트볼 캠핑 밀키트 자취 혼밥 즉석밥 3분요리 오뚜기3분 즉석식품 간편식 즉석음식 간편식품 3분짜장 3분카레 3분미트볼 캠핑요리 간편음식 자취음식 캠핑음식, 단품, 단품
1,390
3분 쇠고기카레 200G, 단품, 단품
1,400
오뚜기 3분 쇠고기짜장 200g 쇠고기카레 200g, 3분쇠고기짜장200g
1,400
3분쇠고기카레 200G
1,420
오뚜기 3분요리 쇠고기카레 200g 1개, 단일 수량
1,450
오뚜기 3분 쇠고기카레 3분 카레 순한맛 약간매운맛 매운맛 200g, 2 오뚜기 3분 카레 순한맛 200g
1,450
오뚜기 3분쇠고기카레200g, 쿠팡 1, 쿠팡 본상품선택
1,460
오뚜기 3분쇠고기카레200g, 상세페이지 참조
1,470
오뚜기 3분 쇠고기카레 200g/즉석식품/국/덮밥/덥밥/소스/카레/짜장/3분요리 식품 > 가공/즉석식품 즉석밥/간편조리 카레/짜장/덮밥 즉석카레, 1
1,530
오뚜기 3분 쇠고기짜장 200g 쇠고기카레 200g, 3분 쇠고기카레
1,570
간단 오뚜기 3분쇠고기짜장200g 커리 인스턴트 캠핑요리 간편식 한끼식사 도시락 야식, 단품, 단품
1,600


...

오랄비 초미세모 크로스액션 차콜 칫솔
2,140
오랄비 크로스액션 프로엑스퍼트 젠틀케어 35모 칫솔, 3개입, 1개
6,050
오랄비 크로스액션 화이트닝 차콜 칫솔, 3개입, 1개
6,620
오랄비 크로스액션 화이트닝 차콜35모 칫솔- 3개입
6,900
오랄비 크로스액션 컴플리트7 부드러운모 3개입 1개
6,950
오랄비 크로스액션 프로엑스퍼트 부드러운 모 35 x 3개입
7,100
오랄비 크로스액션 컴플리트7 35모 칫솔, 3개입, 1개
7,130
오랄비 컴플리트 5웨이 후레쉬 클린 칫솔, 3개입, 1개
7,350
오랄비 크로스액션 그린티 35모 칫솔, 3개입, 1개
7,700
오랄비 크로스액션 플라그가드 40모 칫솔, 1개, 3개입
7,730
오랄비 크로스액션 젠틀케어 칫솔 3개입, 1개
7,900
오랄비 크로스액션 화이트닝 차콜 칫솔, 1개, 3개입
8,330
오랄비 크로스액션 프로 엑스퍼트 향균 35모 칫솔 랜덤발송, 1개, 3개입
8,900
오랄비 크로스액션 컴플리트7 칫솔, 3개입, 1개
8,980
오랄비 크로스액션 프로엑스퍼트 향균 35모 칫솔, 3개입, 1개
9,200
오랄비 크로스액션 35모 칫솔, 5개입, 1개
9,280
오랄비 크로스액션 화이트닝 차콜 35모 칫솔 3개입, 1팩, 3개
9,300
오랄비 크로스액션 프로 엑스퍼트 향균 35모 칫솔 랜덤발송, 3개입, 1개
9,410
오랄비 크로스액션 탁월한 세정력 35모 칫솔 3+2
9,500
오랄비 크로스액션 컴플리트7 35모 칫솔, 1개, 3개입
9,700
오랄비 크로스액션 프로엑스퍼트 컴플리트7 부드러운모/35모 칫솔 3개입, 1팩
9,900
오랄비 칫솔 크로스액션 프로 엑스퍼트 탁월한 세정력 3개입
10,150
오랄비 크로스액션 탁월한 세정력 칫솔, 1개, 3개입
10,700
오랄비 크로스액션 칫솔 35모, 1개, 3개입
10,700
오랄비 크로스액션 항균 칫솔 랜덤발송, 1개입, 6개
10,850
오랄비 크로스액션 항균 칫솔 랜덤발송, 1개입, 6개
10,850
오랄비 크로스액션 탁월

샘표 갈치조림소스200g, 200g, 1개
2,200
국내산 풀치 (실갈치) 300g 500g
4,900
냉동갈치 수입갈치 대서양 모로코 특대 1마리 600g(6토막)
6,500
나래밥상 갈치젓, 180g, 1개
6,560
갈치 풀치 10마리 1kg전후 목포, 10
7,900
엄마애바다 화덕에 구운 갈치구이 150g 전자렌지 2분조리, 1팩
7,910
서풍앤쿡 줄줄이 왕갈치 (냉동), 550g, 1팩
8,000
비비수산 두툼한 A급 왕특대갈치 600g 갈치, 1팩
8,500
한수위 제주바당 갈치살 (냉동), 200g, 1개
8,500
순살 갈치살 (냉동), 500g, 1팩
8,880
새우몰 제주 은갈치 4미(750g이상) 낚시로 잡아 은비늘이 선명한 A급 갈치, 1개
8,900
국내산 순살갈치 200g x 2팩, 1개
9,790
사랑해어 간편한 갈치 조림 (냉동), 400g, 1개
9,900
사랑해어 대서양 손질 갈치 (냉동), 550g, 1팩
10,800
더도톰한 손질 갈치 특대 (냉동), 500g, 1팩
10,880
프리미엄갈치 180g(3토막)*4팩 12토막
10,910
바다자리 제주 손질 은갈치 중 1미, 260g, 1개
11,500
[고래푸드] 국내산 간고등어 6손(12마리), 2.2kg, 12마리
11,900
곰곰 손질 갈치, 700g, 1개
12,350
풀무원 풀스키즈 보성녹차담은 우리갈치 (냉동), 500g(30g내외절단), 1개
14,000
곰곰 손질 갈치 토막, 1kg, 1개
14,990
이츠웰 매실입은 갈치 40~60g 토막 (냉동), 1kg, 1개
15,300
이츠웰 매실입은 갈치 60~80g 토막 (냉동), 1kg, 1개
15,300
서풍 손질 왕 갈치 (냉동), 700g, 1개
15,790
서풍앤쿡 두툼한 왕갈치 (냉동), 600g, 2팩
15,800
씨푸드몰 대서양 손질 갈치 1.4kg (10토막 내외), 1개
15,010
서풍앤쿡 두툼한 왕갈치 (냉동), 1kg, 1봉
15,900
사랑해어 세네갈산 갈치 (냉동), 1kg, 1개
1

손질 대파, 80g, 1봉
990
무농약인증 국내산 대파, 300g, 1봉
1,700
곰곰 국내산 절단 대파, 300g, 1개
1,730
국내산 깐대파, 300g, 1봉
1,810
우주농원 신선한국내산 손질대파, 1팩, 300g
2,350
GAP인증 절단대파, 500g, 1봉
2,370
일등야채 진도대파 1단 부산 강원도 깐대파, 1-1.2kg내외
2,500
국내산 대파 1단 싱싱하고 하부가 긴 하우스대파, 01. 대파 1단
2,500
[당일발송] 새벽들 대파 1단, 1kg내외
2,500
국내산 깐대파, 500g, 1개
2,660
곰곰 국내산 깐대파, 500g, 1봉
2,680
국내산 흙대파, 1kg, 1개
2,680
곰곰 국내산 흙대파, 1kg, 1개
2,680
국내산 절단 손질대파, 500g, 1봉
2,500
곰곰 국내산 절단대파, 500g, 1개
2,500
모들채소 절단대파 300g 1팩
2,900
친환경 제주 절단대파, 300g, 1봉
2,920
곰곰 친환경인증 절단대파, 300g, 1개
3,000
양배추1통, 1개
3,000
[인기용량] 손질 대파 500g, 1팩
3,090
우주농원 국내산 신선한 대파, 1단
3,150
조은먹거리 국내산 대파, 대파 1단, 1kg 내외
3,500
매직 대파, 30g, 1개
4,000
깐대파 국내산, 1kg, 1개
4,100
곰곰 친환경 인증 국내산 흙대파, 1개
4,530
국내산 깐 대파 1kg, 단품
4,600
바로한끼 동결건조 대파, 30g, 1개
4,770
곰곰 국내산 절단대파, 1kg, 1개
4,850
대파 2kg내외 팔팔마켓 국산 파 뿌리대파 한단, kg
4,900
팜피아 냉동 대파 1kg+1kg 총2kg
4,900
대파 국내산 손질대파 2단 2kg
4,900
겟팜스 대파 국내산 반 절단대파 420g 손질대파 당일생산(냉동x), 1팩
4,930
친환경 인증 국내산 절단대파, 500g, 1봉
5,000
친환경 인증 국내산 손질한 흰줄기 대파, 350g, 1개
5,070
국내산 절단 손질대파, 1kg, 1봉
5,

ThankQ 계란 달걀 모양스티로폼/중(15개) 대(10개)/, 백색대짜(5X7cm10개)
1,300
(영일) 아라리 메추리알 15g(1팩-15개), 1개
5,600
양지뜨레 무항생제 인증 햇살 동물복지 유정란 15구, 780g, 1개
7,980
조인 무항생제 인증 일판 대란 30구, 1560g, 1개
8,190
아라리 메추리알15g(3알)x15개 1봉/염지란/훈제란/찐계란/아라리계란 식품 > 신선식품 축산물/계란 계란 구운계란/훈제란, 225g, 1
9,000
노 브랜드 15 개 부활절 달걀 홀더 나무 보관 계란 컵 스탠드 홈, 1개
9,780
수납 정리 함 약 옷장 니트 수건 리모컨 화장대 트레이 팬트리 플라스틱 그리드 계란, 파란색 15개
9,800
뷰티 똥 퍼프 소프트 울다 마이크로 흠집 에그 RT 사절면 건습 투웨이 파운데이션 베이스화장 스폰지 알, 젤리 핑크 퍼프 15 개 _증정 건조
9,900
원래 lol 서프라이즈 인형 색상 변경 계란 색종이 팝 시리즈 드레스 lol 인형 공 어린이 장난감 어린이 크리스마스, (366)1pcs random LOL-15개
9,900
냉장고수납트레이 가정용 15 개 계란 케이스 수납함 상자 룩 알 정리
9,900
EGG 에그 뽑기통 빈 에그타임 계란 캡슐 보관 15개, 에그캡슐
9,900
화장솜용기 돈 아가씨 핏 상큼한 샤워 캡 케이스 8개 B품행과 아름다움 달걀 분장을 위 화장함 스폰지 물에 닿다 변함 대미, 피를 토함 값 15 개 B품 무 케이
10,300
파운데이션 RAINBOWWITHU 포장 가격 미약함 흠집 뷰티 에그 B품 쌍절 면 건습 투웨이 핑크 퍼프, 15 개 미약함 흠집 뷰티 에그
10,600
양지뜨레 무항생제 인증 햇살 자연방사 유정란 15구, 780g, 1개
10,800
계란보관함 전용 냉장고 사이드도어 달걀넣는 수납함 서랍식 신선보존 계란통 수납대받침 접시, T04-블루색(15개포장)
10,900
계란보관함 전용 냉장고 사이드도어 달걀넣는 수납함 서랍식 신선보존 계란통 수납대받침 접시, T03-

친절한한우 우둔 300g600g육회 이유식용 1등급이상 소고기 영주한우 설날 설선물 명절선물 신년선물 새해선물 선물세트, 1개, 육회 물회소스 100g
2,000
[강진완도축협] 한우1등급 불고기 100g
4,800
[강진완도축협] 한우1등급 설도 국거리 100g
5,400
[강진완도축협] 한우1등급 장조림 100g
5,400
[강진완도축협] 한우1등급 양지 국거리 100g
6,400
[강진완도축협] 한우1등급 불고기 100g, 상세 설명 참조
9,800
[강진완도축협] 한우1등급 등심 100g
13,000
브랜드없음 [강진완도축협] 한우1등급 등심 100g, 단품없음
13,100
[강진완도축협] 한우1+등급 등심 100g, 단품
14,400
[강진완도축협] 한우1등급 안심 100g
15,500
[강진완도축협] 한우1+등급 채끝 100g
16,000
[육라운드] 한우1등급 다이어트소고기 한우1+ 안심 구이용 100g 온라인정육점 육라운드 도매
16,500
[강진완도축협] 한우1+등급 안심 100g
17,000
[강진완도축협] 한우1등급 등심 100g, 단품
17,900
안동 한우등심 1등급 100g
18,900
[강진완도축협] 한우1+등급 등심 100g
18,900
[강진완도축협] 한우1등급 채끝 100g
18,900
안동한우 안심 1등급 100g, 1등급 안심
19,000
[강진완도축협] 한우1등급 안심 100g, 상세 설명 참조
20,900
국내산 1등급 이상 육우등심세트(등심400g 치마살100g 제비추리100g) 600g
36,900
농협안심한우 1등급 등심 [100g/300g], 02.한우 등심-300g
52,900
[강진완도축협] 한우1등급 채끝 400g
53,400
기타 기획특가100g 더! 1등급 이상 냉장한우!(등심2 불고기2 국거리1) / 1.kg, 1
69,900
[자연맛남] 마장동한우 명품 구이선물세트 1호 1등급 1kg (등심500g/채끝200g/업진200g/갈비살100g)
134,900
친절한한우 우둔 300g600g육회 이유식용 1등

In [160]:
## 크롤링 결과

# sample_.reverse()
sample_

deque([['머거본 알땅콩(캔) 135g, 1개', '1,890'],
       ['[BIGHAND] 머거본 알땅콩 캔 135g, 1개', '1,970'],
       ['머거본 꿀땅콩 135g 식품 > 신선식품 건과/견과류 견과류 일반땅콩, 1개입', '1,980'],
       ['[머거본] 꿀땅콩 135g, 단품', '2,100'],
       ['상세설명참고 머거본 꿀땅콩 캔 135g', '2,300'],
       ['[머거본] 꿀땅콩 135g 1캔', '2,300'],
       ['머거본 꿀땅콩 135g, 상세페이지 참조', '2,330'],
       ['머거본 꿀땅콩 캔 135g 식품 > 스낵/간식 원물간식 기타원물간식, 1, 1개', '2,390'],
       ['머거본 알땅콩 135g x 1캔', '2,420'],
       ['머거본 꿀땅콩 135g 너트 피넛 캔 안주 허니 맛땅콩, 1N5R_머거본 꿀땅콩 135g', '3,000'],
       ['머거본 허니 꿀땅콩 135g 피넛 견과류 마른 안주 간식 군것질 주전부리', '3,080'],
       ['머거본 원하프꿀땅콩, 135g', '3,310'],
       ['샘표 질러 부드러운 육포 30G, 단품', '3,500'],
       ['(맛나)꿀땅콩 135G/머거본, 1개', '3,530'],
       ['무배_꿀땅콩(머거본 135g)/식당, 1', '3,680'],
       ['머거본 꿀땅콩 135g, 상세페이지 참조', '4,800'],
       ['☞고객대만족↗¶ 견과류 아버지술 소주안주 아빠술 꿀땅콩 맥주안주 머거본 (k‡f*H) 135g, 1개, ♬본 상품 선택하기_Happys™',
        '5,250'],
       ['¶⊙핫딜_A+ 아버지 맥주 술안주 견과류 소주 FK 안주 꿀땅콩 머거본 (Mf!♩x) 135g, 1개, ☞ 해당 상품 선택하기_DoDream™',
        '5,290'],
       ['머거본 꿀

In [161]:
## 크롤링해서 뽑아온 데이터를 DataFrame으로 만듬

present_prod = pd.DataFrame(sample_, columns = ['상품명', '가격'])
present_prod

,상품명,가격
0,"머거본 알땅콩(캔) 135g, 1개","1,890"
1,"[BIGHAND] 머거본 알땅콩 캔 135g, 1개","1,970"
2,"머거본 꿀땅콩 135g 식품 > 신선식품 건과/견과류 견과류 일반땅콩, 1개입","1,980"
3,"[머거본] 꿀땅콩 135g, 단품","2,100"
4,상세설명참고 머거본 꿀땅콩 캔 135g,"2,300"
...,...,...
8618,"깨끗한나라 순수 시그니처 3겹 28m 30롤 화장지 휴지 56191012EA, 위자...","26,250"
8619,KA_깨끗한나라 순수 시그니처 3겹 28m 30롤 화장지 휴지_K/N:3E+8EB1...,"26,250"
8620,"깨끗한나라 순수 시그니처 3겹 화장지 30Mx30롤, 옵션1 - 기본포장 (박스or...","26,850"
8621,찬란한황소_/ 깨끗한나라 순수 시그니처 3겹 28m 30롤 화장지 휴지 #:_454...,"27,580"


In [137]:
present_prod.to_csv('present_prod.csv')

In [192]:
present_prod['상품명']

0                                     머거본 알땅콩(캔) 135g, 1개
1                            [BIGHAND] 머거본 알땅콩 캔 135g, 1개
2             머거본 꿀땅콩 135g 식품 > 신선식품 건과/견과류 견과류 일반땅콩, 1개입
3                                      [머거본] 꿀땅콩 135g, 단품
4                                   상세설명참고 머거본 꿀땅콩 캔 135g
                              ...                        
8618    깨끗한나라 순수 시그니처 3겹 28m 30롤 화장지 휴지 56191012EA, 위자...
8619    KA_깨끗한나라 순수 시그니처 3겹 28m 30롤 화장지 휴지_K/N:3E+8EB1...
8620    깨끗한나라 순수 시그니처 3겹 화장지 30Mx30롤, 옵션1 - 기본포장 (박스or...
8621    찬란한황소_/ 깨끗한나라 순수 시그니처 3겹 28m 30롤 화장지 휴지 #:_454...
8622              깨끗한나라 순수 시그니처 3겹데코 28m 30롤 X 2팩, 단품, 단품
Name: 상품명, Length: 8623, dtype: object

In [214]:
# ## 유니크 품목들의 '()'을 제거한 것

li_list = []
for prodc in products:
    if ')' in prodc:
        
        # print(prodc.split(')')[0].split('('))
#         pr1 = prodc.split(')')[0].split('(')
        
        prodc = prodc.replace('(', " ")
        prodc = prodc.replace(')', "")
        li_list.append(prodc)
    else:
#         print([prodc])
#         pr2 = [prodc]
        
        li_list.append(prodc)
li_list

['머거본 꿀땅콩 135g',
 '허니버터아몬드 210g',
 '백설 소면 900g',
 '옛날국수 소면 900g',
 '고향만두 1.08kg',
 '비비고 왕교자 1.05kg',
 '생야채돼지고기물만두 800g',
 '백설 자른 햇당면 300g',
 '옛날 자른당면 300g',
 '국산콩두부 부침용 380g',
 '국산콩두부 찌개용 380g',
 '행복한콩 부침두부 300g',
 '행복한콩 찌개두부 300g',
 '삼양라면 5개입',
 '신라면 5개입',
 '진라면 순한맛 5개입',
 '곰표 밀가루 중력다목적용 1kg',
 '백설 찰밀가루 1kg',
 '백설 5가지 재료 부침가루 1kg',
 '오뚜기 부침가루 1kg',
 '삼립호떡 미니꿀호떡 16개입',
 '정통크림빵 3개입',
 '보노 콘스프 3개입',
 '보노 포르치니버섯스프 3개입',
 '오뚜기 크림스프 80g',
 '스페셜K 오리지널 480g',
 '켈로그 콘푸로스트 600g',
 '포스트 콘푸라이트 600g',
 '오뚜기 간편 미역국 5개입',
 '오뚜기 간편 북엇국 5개입',
 '3분 쇠고기짜장 200g',
 '3분 쇠고기카레 200g',
 '오뚜기 3분 백세카레 약간 매운맛 200g',
 '청정원 카레여왕 비프카레 160g',
 '오뚜기밥 210g',
 '오뚜기밥 3개입',
 '오뚜기밥 6개입 ',
 '햇반 210g',
 '햇반 3개입',
 '햇반 6개입',
 '동원 양반 쇠고기죽 287.5g',
 '동원 양반 전복죽 287.5g',
 '비비고 소고기죽 280g',
 '비비고 전복죽 280g',
 '농심 신라면 큰사발면 114g',
 '오뚜기 참깨라면 컵 110g',
 '팔도 왕뚜껑 110g',
 '오뚜기 맛있는 쇠고기미역국밥 162g',
 '오뚜기 컵밥 제육덮밥 310g',
 '햇반 컵반 고추장 제육덮밥 250g',
 '햇반 컵반 미역국밥 167g',
 '비비고 사골곰탕 500g',
 '비비고 육개장 500g',
 '오뚜기 옛날 사골곰탕 350g',
 '오뚜기 옛날 육개장 300g',
 '오리

In [199]:
products

['머거본 꿀땅콩(135g)',
 '허니버터아몬드(210g)',
 '백설 소면(900g)',
 '옛날국수 소면(900g)',
 '고향만두(1.08kg)',
 '비비고 왕교자(1.05kg)',
 '생야채돼지고기물만두(800g)',
 '백설 자른 햇당면(300g)',
 '옛날 자른당면(300g)',
 '국산콩두부 부침용(380g)',
 '국산콩두부 찌개용(380g)',
 '행복한콩 부침두부(300g)',
 '행복한콩 찌개두부(300g)',
 '삼양라면(5개입)',
 '신라면(5개입)',
 '진라면 순한맛(5개입)',
 '곰표 밀가루 중력다목적용(1kg)',
 '백설 찰밀가루(1kg)',
 '백설 5가지 재료 부침가루(1kg)',
 '오뚜기 부침가루(1kg)',
 '삼립호떡 미니꿀호떡(16개입)',
 '정통크림빵(3개입)',
 '보노 콘스프(3개입)',
 '보노 포르치니버섯스프(3개입)',
 '오뚜기 크림스프(80g)',
 '스페셜K 오리지널(480g)',
 '켈로그 콘푸로스트(600g)',
 '포스트 콘푸라이트(600g)',
 '오뚜기 간편 미역국(5개입)',
 '오뚜기 간편 북엇국(5개입)',
 '3분 쇠고기짜장(200g)',
 '3분 쇠고기카레(200g)',
 '오뚜기 3분 백세카레 약간 매운맛(200g)',
 '청정원 카레여왕 비프카레(160g)',
 '오뚜기밥(210g)',
 '오뚜기밥(3개입)',
 '오뚜기밥(6개입) ',
 '햇반(210g)',
 '햇반(3개입)',
 '햇반(6개입)',
 '동원 양반 쇠고기죽(287.5g)',
 '동원 양반 전복죽(287.5g)',
 '비비고 소고기죽(280g)',
 '비비고 전복죽(280g)',
 '농심 신라면 큰사발면(114g)',
 '오뚜기 참깨라면 컵(110g)',
 '팔도 왕뚜껑(110g)',
 '오뚜기 맛있는 쇠고기미역국밥(162g)',
 '오뚜기 컵밥 제육덮밥(310g)',
 '햇반 컵반 고추장 제육덮밥(250g)',
 '햇반 컵반 미역국밥(167g)',
 '비비고 사골곰탕(500g)',
 '비비고 육개장(

In [206]:
sorted(li_list)

['3M 후레쉬 슬라이드 지퍼백 중형 21cm x 16cm x 6cm',
 '3분 쇠고기짜장 200g',
 '3분 쇠고기카레 200g',
 'CJ 더안심 건강란 대란 15개',
 'CJ 비비고 왕교자 910g',
 'CJ 알짜란 대란 15개',
 'CJ 행복한콩 국산콩 콩나물 280g',
 'JB 뉴 베드타임 로션 500ml',
 'PACE6 스타일 면도날 4개',
 '가그린 오리지널 750ml',
 '갈치 냉동, 100g',
 '갈치 생물, 100g',
 '게토레이 레몬 600ml',
 '고등어 생물, 300~500g',
 '고향만두 1.08kg',
 '곰표 밀가루 중력다목적용 1kg',
 '국산아작아작콩나물 350g',
 '국산콩두부 부침용 380g',
 '국산콩두부 찌개용 380g',
 '국순당 생막걸리 750ml',
 '국순당 우국생 750ml',
 '그릴후랑크 600g',
 '깨끗한나라 순수 시그니쳐 3겹 30롤',
 '남양 17차 티백 80개입',
 '남양유업 맛있는우유GT 1L',
 '농심 새우깡 90g',
 '농심 신라면 큰사발면 114g',
 '다우니 고농축 섬유유연제 엑스퍼트 2L',
 '닥터유 에너지바 40g',
 '닥터유 에너지바 4개입',
 '달콤한 베지밀 비 16팩',
 '달콤한 삼육두유 B',
 '당근 흙당근, 100g',
 '대림 게맛살 큰잔치 300g',
 '대림선어묵 부산어묵 300g',
 '대파 흙대파, 500~800g',
 '덴마크 오리지널 치즈 14매',
 '델몬트 오렌지 100 1.5L',
 '도브 뷰티 너리싱 바디워시 1kg',
 '도브 뷰티바 4개',
 '도브 센스티브 스킨 바디워시 1kg',
 '동서 아카시아꿀 600g',
 '동서벌꿀 600g',
 '동서현미녹차 100개입',
 '동원 싱싱맛살 실속 500g',
 '동원 양반 쇠고기죽 287.5g',
 '동원 양반 전복죽 287.5g',
 '동원참치 라이트스탠다드 150g',
 '동원참치 라이트스탠다드 4캔',
 '돼지고기 목살 100g',
 '돼지고기 삼겹살 10

In [215]:
## 괄호를 제거한 유니크 품목들을 문자열로 전환


'|'.join(li_list)


'머거본 꿀땅콩 135g|허니버터아몬드 210g|백설 소면 900g|옛날국수 소면 900g|고향만두 1.08kg|비비고 왕교자 1.05kg|생야채돼지고기물만두 800g|백설 자른 햇당면 300g|옛날 자른당면 300g|국산콩두부 부침용 380g|국산콩두부 찌개용 380g|행복한콩 부침두부 300g|행복한콩 찌개두부 300g|삼양라면 5개입|신라면 5개입|진라면 순한맛 5개입|곰표 밀가루 중력다목적용 1kg|백설 찰밀가루 1kg|백설 5가지 재료 부침가루 1kg|오뚜기 부침가루 1kg|삼립호떡 미니꿀호떡 16개입|정통크림빵 3개입|보노 콘스프 3개입|보노 포르치니버섯스프 3개입|오뚜기 크림스프 80g|스페셜K 오리지널 480g|켈로그 콘푸로스트 600g|포스트 콘푸라이트 600g|오뚜기 간편 미역국 5개입|오뚜기 간편 북엇국 5개입|3분 쇠고기짜장 200g|3분 쇠고기카레 200g|오뚜기 3분 백세카레 약간 매운맛 200g|청정원 카레여왕 비프카레 160g|오뚜기밥 210g|오뚜기밥 3개입|오뚜기밥 6개입 |햇반 210g|햇반 3개입|햇반 6개입|동원 양반 쇠고기죽 287.5g|동원 양반 전복죽 287.5g|비비고 소고기죽 280g|비비고 전복죽 280g|농심 신라면 큰사발면 114g|오뚜기 참깨라면 컵 110g|팔도 왕뚜껑 110g|오뚜기 맛있는 쇠고기미역국밥 162g|오뚜기 컵밥 제육덮밥 310g|햇반 컵반 고추장 제육덮밥 250g|햇반 컵반 미역국밥 167g|비비고 사골곰탕 500g|비비고 육개장 500g|오뚜기 옛날 사골곰탕 350g|오뚜기 옛날 육개장 300g|오리온 포카칩 오리지널 66g|포테토칩 오리지날 125g|롯데 자일리톨 오리지날 97g|빙그레 참붕어 싸만코 150ml|아이비 270g|참크래커 280g|돼지바 70ml|메로나 75ml|해태 바밤바 63ml|롯데 목캔디 허브민트 217g|오리온 통아몬드 캔디 90g|농심 새우깡 90g|롯데 월드콘 바닐라 160ml|투게더 오리지널 바닐라 900ml|해태 부라보콘 화이트바닐라 140ml|닥터유 에너

In [216]:
## 유니크 품목이 포함돼있는 상품명(크롤링 해서 뽑아온 유니크 품목당 모든 제품들)을 추출

bool_idx = present_prod['상품명'].str.contains('|'.join(li_list))

In [217]:
present_prod[bool_idx]

,상품명,가격
2,"머거본 꿀땅콩 135g 식품 > 신선식품 건과/견과류 견과류 일반땅콩, 1개입","1,980"
6,"머거본 꿀땅콩 135g, 상세페이지 참조","2,330"
9,"머거본 꿀땅콩 135g 너트 피넛 캔 안주 허니 맛땅콩, 1N5R_머거본 꿀땅콩 135g","3,000"
15,"머거본 꿀땅콩 135g, 상세페이지 참조","4,800"
18,"머거본 꿀땅콩 135g, 3개입","5,310"
...,...,...
8233,"삼양 원조 삼양라면 5개입, 15개","7,540"
8515,"CJ 비비고 왕교자 1.05kg x 2봉+수제고기만두 400g x 2봉, 없음, 상...","27,320"
8517,과자 해태 오예스 12개입 336g,"4,410"
8530,"해태 오예스 12개입 (360g), 옵션2 - 신선포장 (아이스박스포장)","9,170"


In [231]:
present_prod[bool_idx]['상품명'].str.split('g')

2          [머거본 꿀땅콩 135,  식품 > 신선식품 건과/견과류 견과류 일반땅콩, 1개입]
6                               [머거본 꿀땅콩 135, , 상세페이지 참조]
9       [머거본 꿀땅콩 135,  너트 피넛 캔 안주 허니 맛땅콩, 1N5R_머거본 꿀땅콩...
15                              [머거본 꿀땅콩 135, , 상세페이지 참조]
18                                   [머거본 꿀땅콩 135, , 3개입]
                              ...                        
8233                                [삼양 원조 삼양라면 5개입, 15개]
8515    [CJ 비비고 왕교자 1.05k,  x 2봉+수제고기만두 400,  x 2봉, 없음...
8517                               [과자 해태 오예스 12개입 336, ]
8530          [해태 오예스 12개입 (360, ), 옵션2 - 신선포장 (아이스박스포장)]
8533    [해태 오예스 12개입 360,  +오리온 초코파이 468,  초콜릿 케이크 파이 ...
Name: 상품명, Length: 703, dtype: object

In [169]:
present_prod[bool_idx]['상품명'].apply(lambda x : x if prod in x else np.nan)

2      NaN
6      NaN
9      NaN
15     NaN
18     NaN
        ..
8233   NaN
8515   NaN
8517   NaN
8530   NaN
8533   NaN
Name: 상품명, Length: 703, dtype: float64

In [181]:
def prod_split(x):
    try:
        for pro in prods:
            return x.split(pro)[0]
    except:
        return x

In [182]:
prod_ar = present_prod[bool_idx]
prod_ar.상품명.apply(prod_split)

2             머거본 꿀땅콩 135g 식품 > 신선식품 건과/견과류 견과류 일반땅콩, 1개입
6                                  머거본 꿀땅콩 135g, 상세페이지 참조
9       머거본 꿀땅콩 135g 너트 피넛 캔 안주 허니 맛땅콩, 1N5R_머거본 꿀땅콩 135g
15                                 머거본 꿀땅콩 135g, 상세페이지 참조
18                                      머거본 꿀땅콩 135g, 3개입
                              ...                        
8233                                  삼양 원조 삼양라면 5개입, 15개
8515    CJ 비비고 왕교자 1.05kg x 2봉+수제고기만두 400g x 2봉, 없음, 상...
8517                                  과자 해태 오예스 12개입 336g
8530             해태 오예스 12개입 (360g), 옵션2 - 신선포장 (아이스박스포장)
8533    해태 오예스 12개입 360g +오리온 초코파이 468g 초콜릿 케이크 파이 묶음,...
Name: 상품명, Length: 703, dtype: object

In [145]:
prod_ar.iloc[3]

상품명    CJ 비비고 왕교자 1.05kg x 2봉+수제고기만두 400g x 2봉, 없음, 상...
가격                                                27,320
Name: 107, dtype: object

In [6]:
# for prod in prods:
#     url = 'https://www.coupang.com/np/search?rocketAll=false&q={}&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&searchProductCount=326&component=&rating=0&sorter=salePriceAsc&listSize=36'
#     url = url.format(prod)
    
#     headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'}
    
#     res = requests.get(url, headers = headers)
#     soup = BeautifulSoup(res.text, 'html.parser')
    
#     lis = soup.select(' a > dl > dd > div')
    
#     for li in lis:
#         if prod in li.select('div.name')[0].text:
#             print(li.select('div.name')[0].text)
#             print(li.select('strong.price-value')[0].text, '원')
#             print('===========')
#             break
            
    

vono 보노스프(3개입) x 1개 /보노스프 콘스프/간편식/간편한아침/간편스프/즉석스프, 01 보노 콘스프(3개입)
2,550 원
농심 새우깡(90g)
1,510 원
주전부리몰 오리온 닥터유 에너지바(4개입) x 5통(+밀크츄 2봉포함), 1세트
19,900 원
청정원 오리엔탈 드레싱 325G, 옵션2 - 안전포장 플러스
5,000 원
백설 진한참기름450ml, 1개
5,130 원
삼육두유 삼육식품 달콤한 삼육두유 B 190ML 24개입, 1
19,500 원
[샘플사료] 밥이보약 CAT 걱정없는 헤어볼 40g 400g, [41-6] 40g
1,100 원
밥이보약 튼튼한관절2Kg(강아지용)
22,500 원
크리넥스 KF94 4단 마스크 대형 1매, 상세페이지 참조
1,040 원
해태 오예스(12개입) 360g + 오뜨 쇼콜라(6개입) 150g 초콜릿 파이 초코 케익 묶음 세트, 1세트
7,980 원


In [2]:
# url = 'https://www.coupang.com/np/search?rocketAll=false&q=%EC%82%BC%EC%96%91%EB%9D%BC%EB%A9%B4%285%EA%B0%9C%EC%9E%85%29&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&searchProductCount=94&component=&rating=0&sorter=salePriceAsc&listSize=36'
    

    

In [3]:
# headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'}

In [4]:
# res = requests.get(url, headers = headers)


In [5]:
# soup = BeautifulSoup(res.text, 'html.parser')

In [6]:
# lis = soup.select(' a > dl > dd > div')
# lis

[<div class="descriptions-inner"><div class="badges"> </div><div class="name">삼양라면 오리지널 멀티팩(5개입) x 1봉/봉지라면</div> <div class="price-area"> <div class="price-wrap"><div class="price"> <span class="price-info"> <span class="instant-discount-rate">43%</span> <del class="base-price"> 5,000  </del> </span> <em class="sale"> <strong class="price-value">2,800</strong>원     </em> </div><!-- Free Shipping Badge -->
 <div class="delivery"><span class="arrival-info"> <em style="color:#111111">토요일 2/19  </em> <em style="color:#111111">도착 예정 </em> </span></div> </div> </div><div class="other-info"><div class="rating-star"><span class="star"><em class="rating" style="width:100%">5.0</em></span> <span class="rating-total-count">(1)</span></div></div><div class="benefit-badges"> <div class="reward-cash-badge"><div class="reward-cash-badge__inr"><img alt="" class="reward-cash-ico" src="//image6.coupangcdn.com/image/badges/cashback/web/list-cash-icon@2x.png"/> <span class="reward-cash-txt">최대 140원 적립</sp

In [13]:
# for li in lis:
#     print(li.select('strong.price-value')[0].text)

3,290
3,660
6,300
7,500
7,510
8,700
8,700
9,040
10,450
11,100
12,000
12,330
12,790
14,900
15,670
19,900
30,800
50,900
11,090
